 # Attention-Based Spillover Table Using Feature-Level Attention



 This version creates a custom attention mechanism across features (vech dimensions)

 and extracts directional spillover strength from learned attention weights.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import matplotlib.pyplot as plt

# Load PIT-transformed vech data
option = 'europe'
pit_vech = pd.read_parquet(f"parquet_files/pit_transformed_vech_{option}.parquet")
pit_vech = pit_vech.dropna()
columns = pit_vech.columns
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(pit_vech)

# Sequence creation
def create_sequences(data, window):
    X, Y = [], []
    for i in range(len(data) - window):
        X.append(data[i:i+window])
        Y.append(data[i+window])
    return np.array(X), np.array(Y)

window = 10
X, Y = create_sequences(data_scaled, window)
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32)
dataset = TensorDataset(X_tensor, Y_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


In [ ]:
# Custom Attention Layer Across Features
class FeatureAttentionModel(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.attn = nn.Linear(num_features, num_features, bias=False)
        self.out = nn.Linear(num_features, num_features)

    def forward(self, x):
        # x shape: [batch, time, features]
        x_last = x[:, -1, :]  # use last time step
        weights = torch.softmax(self.attn.weight, dim=1)  # feature-to-feature attention
        attended = torch.matmul(x_last, weights.T)
        out = self.out(attended)
        return out, weights.detach()

model = FeatureAttentionModel(num_features=X.shape[2])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()




 ## Step 2: Train Attention Model and Collect Attention Matrices

In [ ]:
epochs = 10
all_attention_matrices = []

for epoch in range(epochs):
    for xb, yb in dataloader:
        pred, weights = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Capture attention matrices (one per batch)
        all_attention_matrices.append(weights.detach().cpu().numpy())

    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item():.4f}")


 ## Step 3: Compute Average Attention Matrix (Feature-to-Feature)

In [ ]:
# Stack and average attention weights
attn_array = np.stack(all_attention_matrices)  # shape: [num_batches, features, features]
attn_avg = attn_array.mean(axis=0)  # shape: [features, features]

# Normalize rows to sum to 1 (spillover matrix)
spillover_matrix = attn_avg / attn_avg.sum(axis=1, keepdims=True) * 100


 ## Step 4: Construct Spillover Table (TO, FROM, NET)

In [ ]:
directional_to = spillover_matrix.sum(axis=1) - np.diag(spillover_matrix)
directional_from = spillover_matrix.sum(axis=0) - np.diag(spillover_matrix)
net_directional = directional_to - directional_from

spillover_table = pd.DataFrame(spillover_matrix,
                               index=columns,
                               columns=columns)
spillover_table["Directional FROM others"] = directional_from
spillover_table.loc["Directional TO others"] = list(directional_to) + [directional_to.sum()]
spillover_table.loc["NET Directional"] = list(net_directional) + [np.nan]


 ## Step 5: Display and Save Spillover Table

In [ ]:
from IPython.display import display, Markdown

display(spillover_table.round(2))

caption = f"""
**Table X**: Attention-Based Spillover Table (Feature-Level Weights)

This table is derived from a neural attention model trained to forecast vectorized PIT-transformed realized variances and covariances. 
The attention matrix is interpreted as a nonlinear spillover mapping between variables. Rows are normalized to sum to 100%.

**Directional spillovers (TO / FROM / NET)** reflect learned influences based on attention weights aggregated over all training samples.
"""
display(Markdown(caption))

# Save table
spillover_table.to_parquet(f"parquet_files/attention_spillover_table_{option}.parquet")
